# Cell Type Assignment with Garnett

Garnett performs hierarchical cell type assignment, information can be found here: https://cole-trapnell-lab.github.io/garnett/docs_m3/

In [1]:
# Get environment variables
classifier <- Sys.getenv("SNAKEMAKE_CLASSIFIER")
data <- Sys.getenv("SNAKEMAKE_H5AD_INPUT")
output_file <- Sys.getenv("SNAKEMAKE_OUTPUT_FILE")  # Output csv mapping barcodes to cell type data
organsim <- Sys.getenv("SNAKEMAKE_ORGANISM")  # Must be either "human" or "mouse"
# Check that the file exists
if (!file.exists(classifier)) {
  stop(paste("Classifier file does not exist:", classifier))
}

if (!file.exists(data)) {
  stop(paste("Input data file does not exist:", data))
}

cat(paste("Classifier file:", classifier, "\n"))
cat(paste("Input data file:", data, "\n"))
cat(paste("Output file:", output_file, "\n"))
cat(paste("Organism:", organsim, "\n"))

ERROR: Error: Marker list file does not exist: 


In [ ]:
library(garnett)
# Load the gene database according to the organism
if (organsim == "human") {
  gene_db <- require(org.Hs.eg.db)
} else if (organsim == "mouse") {
  gene_db <- require(org.Mm.eg.db)
} else {
  stop("Organism must be either 'human' or 'mouse'.")
}

Load the marker list and classifier

In [ ]:
classifier <- readRDS(classifier)
classifier

Load the single cell data as a SingleCellExperiment object and then convert to a cell_data_set (CDS) object

In [ ]:
library(zellkonverter)
setZellkonverterVerbose(TRUE)
adata <- readH5AD(data)
adata

In [ ]:
library(monocle3)
gene_meta = as.data.frame(rowData(adata))
if (!"gene_short_name" %in% colnames(gene_meta)) {
    gene_meta$gene_short_name <- rownames(gene_meta)
}
cds <- new_cell_data_set(
    expression_data = counts(adata),
    cell_metadata = as.data.drame(colData(adata)),
    gene_metadata = gene_meta
)
head(cds)

Classify cells

In [ ]:
cds <- classify_cells(
    cds, classifier,
    db = gene_db,
    cluster_extend = TRUE,
    cds_gene_id_type = "SYMBOL"
)
head(pData(cds))

In [ ]:
table(pData(cds)$cell_type)

In [ ]:
table(pData(cds)$cluster_ext_type)

Save the cell type assignments to a CSV file

In [ ]:
df <- pData(cds)[, c("garnett_cluster", "cell_type", "cluster_ext_type")]
df$barcode <- rownames(df)
df

In [ ]:
write.csv(
    df,
    file = output_file,
    row.names = FALSE,
)

Session Information

In [ ]:
sessionInfo()